# 1. gradio로 애플, 마이크로소프트, 테슬라, 엔비디아 주식 - yahoo finance에서 2014-전날까지 불러오기
# 2. 데이터 표시
# 3. 종가를 타겟 변수로 지정
# 4. AutoGluon을 이용해 시계열 분석
# 5. 30일 예측 결과 출력, 시각화

In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
import yfinance as yf
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [31]:
import pandas as pd
import gradio as gr
import yfinance as yf
import matplotlib.pyplot as plt
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from datetime import datetime, timedelta

# 종목 리스트
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]

# Yahoo Finance에서 주식 데이터 가져오기 (2014년 1월 1일부터 어제까지)
def fetch_stock_data(selected_ticker):
    print(f"Fetching stock data for {selected_ticker}...")
    start_date = "2014-01-01"  # 2014년 1월 1일부터 데이터 가져오기
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')  # 어제 날짜까지
    data = yf.download(selected_ticker, start=start_date, end=end_date)
    print(f"Downloaded {len(data)} rows of data for {selected_ticker}.")
    return data

# AutoGluon-TimeSeries를 통해 시계열 예측 수행
def automl_time_series_forecasting(data, time_limit=300, forecast_length=30):
    print("Starting AutoGluon time series forecasting...")
    
    # 데이터프레임에서 datetime 타입으로 변환
    data.index = pd.to_datetime(data.index)
    
    # 학습 데이터 구성 (TimeSeriesDataFrame 형식으로 변환)
    train_data = pd.DataFrame({
        "timestamp": data.index,  # 시계열 데이터의 타임스탬프
        "target": data['Close'],  # 예측할 종가 값
        "item_id": "stock"  # 단일 종목이므로 동일 ID로 설정
    }).reset_index(drop=True)  # 인덱스를 초기화해줘야 AutoGluon이 제대로 학습 가능

    print(f"Training data prepared with {len(train_data)} rows.")

    # TimeSeriesDataFrame으로 변환
    train_data_ts = TimeSeriesDataFrame(train_data)

    # AutoGluon TimeSeries 모델 생성 (freq 설정 추가)
    predictor = TimeSeriesPredictor(
        prediction_length=forecast_length,  # 예측할 기간 설정 (30일)
        freq="1D"  # 주기: 일 단위
    )

    # 모델 학습 (time_limit은 fit()에서 설정)
    predictor.fit(train_data_ts, time_limit=time_limit)

    print("Model training completed. Predicting future values...")

    # 예측할 데이터 생성 (미래 데이터 예측)
    future_forecast = predictor.predict(train_data_ts)  # AutoGluon은 자체적으로 미래 데이터를 예측함
    print(f"Forecasting completed. Generated {len(future_forecast)} predicted values.")
    
    return future_forecast

# 시각화 함수: 예측 결과
def plot_forecast(data, forecast):
    print("Plotting forecast results...")
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data['Close'], label='Actual', marker='o')
    plt.plot(forecast.index, forecast['mean'], label='Predicted', linestyle='--', marker='x')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Actual vs Predicted Close Prices')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('forecast_plot.png')
    plt.close()
    print("Plot saved as 'forecast_plot.png'.")
    return 'forecast_plot.png'  # 이미지 경로 반환

# 매수/매도 결정 함수 (간단한 가격 비교로 결정)
def make_trade_decision(forecast):
    print("Making trade decision...")
    last_predicted_price = forecast['mean'].iloc[-1]  # 마지막 예측된 종가

    # 간단한 조건으로 매수/매도 결정
    if last_predicted_price > forecast['mean'].iloc[-2]:
        print("Trade decision: Buy signal.")
        return "매수 신호: 예측된 가격이 상승할 것으로 보입니다."
    elif last_predicted_price < forecast['mean'].iloc[-2]:
        print("Trade decision: Sell signal.")
        return "매도 신호: 예측된 가격이 하락할 것으로 보입니다."
    else:
        print("Trade decision: Hold signal.")
        return "유지 신호: 예측된 가격에 큰 변화가 없습니다."

# 분석 함수
def analyze_stock(selected_ticker):
    print(f"Starting analysis for {selected_ticker}...")
    
    stock_data = fetch_stock_data(selected_ticker)
    
    # AutoGluon을 사용하여 30일 예측
    forecast = automl_time_series_forecasting(stock_data)
    
    # 예측 결과 시각화
    forecast_plot = plot_forecast(stock_data, forecast)
    
    # 매수/매도 신호 계산
    trade_signal = make_trade_decision(forecast)

    print(f"Analysis for {selected_ticker} completed.")
    return forecast_plot, trade_signal  # 이미지 경로와 매수/매도 신호 반환

# Gradio 인터페이스 구성
with gr.Blocks() as interface:
    with gr.Row():
        with gr.Column():
            # 종목 선택 드롭다운 (기본값 설정)
            stock_selector = gr.Dropdown(choices=tickers, label="주식 종목 선택", value="AAPL")  # 기본값 AAPL로 설정
            run_button = gr.Button("30일 예측 및 시각화")

    with gr.Row():
        # 예측 결과 시각화 이미지 출력
        with gr.Column():
            forecast_image_output = gr.Image(label="실제값 VS 예측값")
        
    with gr.Row():
        # 매수/매도 신호 출력
        trade_signal_output = gr.Textbox(label="매수/매도 신호")

    # 버튼 클릭 시 분석 실행
    def run_analysis(selected_ticker):
        try:
            forecast_plot, trade_signal = analyze_stock(selected_ticker)
            return forecast_plot, trade_signal  # 이미지 경로와 신호를 반환
        except Exception as e:
            print(f"Error during analysis: {e}")
            return None, f"분석 중 오류 발생: {e}"  # 오류 발생 시 None 반환, 오류 메시지 반환
    
    # 버튼 클릭 시 실행 및 업데이트
    run_button.click(
        run_analysis,
        inputs=[stock_selector],
        outputs=[forecast_image_output, trade_signal_output]  # 이미지를 먼저 반환하고, 텍스트 신호 반환
    )

# Gradio 앱 실행
print("Launching Gradio interface...")
interface.launch()


Launching Gradio interface...


INFO:httpx:HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[*********************100%***********************]  1 of 1 completed
Frequency '1D' stored as 'D'
Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to 'AutogluonModels/ag-20241017_090338'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
GPU Count:          0
Memory Avail:       6.28 GB / 15.31 GB (41.0%)
Disk Space Avail:   27.16 GB / 237.85 GB (11.4%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 30,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_mode

Starting analysis for AAPL...
Fetching stock data for AAPL...
Downloaded 2715 rows of data for AAPL.
Starting AutoGluon time series forecasting...
Training data prepared with 2715 rows.


Provided train_data has 3940 rows (NaN fraction=31.1%), 1 time series. Median time series length is 3940 (min=3940, max=3940). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-10-17 18:03:40
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. Training for up to 22.9s of the 298.3s of remaining time.
	-0.0178       = Validation score (-WQL)
	0.03    s     = Training runtime
	1.90    s     = Validation (prediction) runtime
Training timeseries model RecursiveTabular. Training for up to 24.7s of the 296.3s of remaining

Model training completed. Predicting future values...


Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


Forecasting completed. Generated 30 predicted values.
Plotting forecast results...
Error during analysis: 'value' must be an instance of str or bytes, not a tuple


In [30]:
interface.close()

Closing server running on port: 7861
